# How do we make software understand language?

## Analogy with language tests
Fill in the missing word

## Encoding
Every string is already encoded character per character

In [1]:
for char in ("abcdefghijklmnopqrstuvwxyz"):
  print("UTF-8 code for char " + char + " is: " + str(ord(char)))

UTF-8 code for char a is: 97
UTF-8 code for char b is: 98
UTF-8 code for char c is: 99
UTF-8 code for char d is: 100
UTF-8 code for char e is: 101
UTF-8 code for char f is: 102
UTF-8 code for char g is: 103
UTF-8 code for char h is: 104
UTF-8 code for char i is: 105
UTF-8 code for char j is: 106
UTF-8 code for char k is: 107
UTF-8 code for char l is: 108
UTF-8 code for char m is: 109
UTF-8 code for char n is: 110
UTF-8 code for char o is: 111
UTF-8 code for char p is: 112
UTF-8 code for char q is: 113
UTF-8 code for char r is: 114
UTF-8 code for char s is: 115
UTF-8 code for char t is: 116
UTF-8 code for char u is: 117
UTF-8 code for char v is: 118
UTF-8 code for char w is: 119
UTF-8 code for char x is: 120
UTF-8 code for char y is: 121
UTF-8 code for char z is: 122


## Install dependencies

In [28]:
!pip install tokenizers==0.12.1 transformers==4.21.3 pandas sklearn datasets
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://download.pytorch.org/whl/cu118, https://us-python.pkg.dev/colab-wheels/public/simple/


## Load in data

In [3]:
import pandas as pd

In [4]:
df_dutch = pd.read_csv("dutch.csv", delimiter=";")

In [5]:
df_dutch

,"Spanje is met ingang van vandaag voorzitter van de EU. De Zweedse premier Fredrik Reinfeldt heeft het stokje, formeel om middernacht, overgedragen aan zijn Spaanse collega JoseÌ Luis Rodriguez Zapatero. Spanje is het eerste land dat het roulerend voorzitterschap overneemt onder het Verdrag van Lissabon, dat op 1 december in werking is getreden. Nieuwe functies De rol van het voorzitterschap is met het in werking treden van het Verdrag van Lissabon veranderd. Voortaan zal de Belg Herman van Rompuy de vergaderingen van de Europese Raad voorzitten. Van Rompuy vertegenwoordigt de EU ook internationaal, samen met de Britse Catherine Ashton. Zij is de buitenlandminister van de EU, ook een nieuwe functie. Spanje heeft het economisch herstel hoog op de agenda van de Europese Unie gezet. Van Rompuy organiseert volgende maand een extra EU-top over de aanpak van de economische crisis. Geslaagd De Zweden mogen terugzien op een geslaagd voorzitterschap. In het afgelopen half jaar kwam het nieuwe Verdrag er, werden er topbenoemingen geregeld en de kredietcrisis bestreden. Alleen de klimaattop in Kopenhagen was een project dat minder succesvol werd afgesloten."
0,Vijf werknemers van het omstreden Amerikaanse ...
1,Het Oud en Nieuwfeest op het Museumplein in Am...
2,President Obama heeft de eerste rapporten gekr...
3,In de hele wereld is het nieuwe jaar feestelij...
4,De hoofdprijs van de oudejaarstrekking van de ...
...,...
9972,Een Chinese trein heeft het snelheidsrecord ge...
9973,Een universiteit in de Amerikaanse staat Texas...
9974,In Brussel demonstreren tienduizenden mensen v...
9975,De NS wil het papieren spoorboekje afschaffen....


## Split by space

In [6]:
sentence = df_dutch.iloc[0]
sentence = sentence[0]

In [7]:
sentence.split(" ")[:10]

['Vijf',
 'werknemers',
 'van',
 'het',
 'omstreden',
 'Amerikaanse',
 'beveilingingsbedrijf',
 'Blackwater',
 'gaan',
 'vrijuit']

## Tokenizers 
### Byte pair encoding

In [8]:
from tokenizers import ByteLevelBPETokenizer

tokenizer = ByteLevelBPETokenizer()

tokenizer.train(files="dutch.csv", vocab_size=1000, min_frequency=3, 
                show_progress=True,
                special_tokens=["<s>", "<pad>", "</s>", "<unk>", "<mask>"])

In [11]:
import os

if not os.path.exists("tokenizer"): os.mkdir("tokenizer")
tokenizer.save_model("tokenizer")

['tokenizer/vocab.json', 'tokenizer/merges.txt']

In [12]:
from transformers import RobertaTokenizerFast

tokenizer = RobertaTokenizerFast.from_pretrained("tokenizer", max_len=512)

In [13]:
tokens = tokenizer.encode(sentence[:100])
print(sentence[:100])
print(tokens)

Vijf werknemers van het omstreden Amerikaanse beveilingingsbedrijf Blackwater gaan vrijuit voor hun 
[0, 58, 766, 733, 82, 397, 364, 284, 293, 356, 274, 300, 295, 848, 327, 600, 326, 310, 632, 468, 72, 396, 74, 354, 80, 722, 79, 91, 767, 673, 835, 343, 329, 606, 225, 2]


In [14]:
print(tokenizer.decode([0, 58, 766, 733, 82, 397, 364, 284, 293, 356, 274, 300, 295, 848, 327, 600, 326, 310, 632, 468, 72, 396, 74, 354, 80, 722, 79, 91, 767, 673, 835, 343, 329, 606, 225, 2]))

<s>Vijf werknemers van het omstreden Amerikaanse beveilingingsbedrijf Blackwater gaan vrijuit voor hun </s>


## Configure RoBERTa model

In [15]:
from transformers import RobertaConfig
from transformers import RobertaForMaskedLM

# Set a configuration for our RoBERTa model
config = RobertaConfig(
    vocab_size=50265,
    max_position_embeddings=514,
    num_attention_heads=12,
    num_hidden_layers=6,
    type_vocab_size=1,
)
# Initialize the model from a configuration without pretrained weights
model = RobertaForMaskedLM(config=config)
print('Num parameters: ',model.num_parameters())

Num parameters:  82170201


In [16]:
from sklearn.model_selection import train_test_split
df_train, df_test = train_test_split(df_dutch, test_size=0.2)

In [17]:
print(len(df_dutch), len(df_train), len(df_test))

9977 7981 1996


In [18]:
from transformers import DataCollatorForLanguageModeling

# Define the Data Collator
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer, mlm=True, mlm_probability=0.15,
)

In [19]:
!pip install --upgrade accelerate

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [20]:
import torch

train_on_gpu = torch.cuda.is_available()

if not train_on_gpu:
    print('CUDA is not available.  Training on CPU ...')
else:
    c = torch.cuda.device_count()
    print(f"CUDA is available!\nNumber of gpu's detected: {c}" )
    print('Training on GPU ...\n\n')
    for i in range(c):
        print(f'Device name {i}== {torch.cuda.get_device_name(i)}')

CUDA is available!
Number of gpu's detected: 1
Training on GPU ...


Device name 0== Tesla T4


In [26]:
def encode(batch):
    return tokenizer(batch['labels'], padding="max_length", truncation=True, max_length=512,return_tensors="pt")

In [27]:
from datasets import Dataset
ds_traiqs = Dataset.from_pandas(df_trai)
df_train.set_transform(encode)
df_test.set_transform(encode)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <cell line: 1>:1                                                                              │
│                                                                                                  │
│ /usr/local/lib/python3.10/dist-packages/pandas/core/generic.py:5902 in __getattr__               │
│                                                                                                  │
│    5899 │   │   │   and self._info_axis._can_hold_identifiers_and_holds_name(name)               │
│    5900 │   │   ):                                                                               │
│    5901 │   │   │   return self[name]                                                            │
│ ❱  5902 │   │   return object.__getattribute__(self, name)                                       │
│    5903 │                                                                                        │
│    5904 │   def __setattr__(self, name: str, value) -> None:                                     │
│    5905 │   │   """                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'DataFrame' object has no attribute 'set_transform'

In [24]:
from transformers import TrainingArguments, Trainer

if not os.path.exists("models"): os.mkdir("models")
if not os.path.exists("models/roberta"): os.mkdir("models/roberta")

training_args = TrainingArguments(
    output_dir='./models/roberta',
    overwrite_output_dir=True,
    evaluation_strategy = 'steps',
    num_train_epochs=1000,
    learning_rate=1e-5,
    lr_scheduler_type="constant",
    weight_decay=0.01,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    save_steps=2048,
    eval_steps=2048,
    save_total_limit=3,
    ignore_data_skip=True,
    gradient_accumulation_steps=4,
    gradient_checkpointing=True,
    fp16=True
)
# Create the trainer for our model
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=df_train,
    eval_dataset=df_test,
    #prediction_loss_only=True,
)

Using cuda_amp half precision backend
